# Task 1 Preguntas teóricas

Responda las siguientes preguntas de forma clara y concisa, pueden subir un PDF o bien dentro del mismo Jupyter Notebook.

1. Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

El algoritmo AC-3, *signfica Arc Consistency 3*. Es un método que se usa para poder cumplir con la consistencia (Arc consistency) en un problema de satisfacción de restricciones CSP. El objetivo es eliminar valores problemáticos o bien inconsistentes del dominio de las variables, por ejemplo eliminar variables en un arco entre un par de variables (x,y), que no cumplen las restricciones entre "x" y "y". Esto ayuda a reducir el espacio de búsqueda antes de aplicar algoritmos. Usualmente el AC-3 forma parte del procesamiento del entorno para simplificar el problema de CSP antes de aplicar Backtracking Search por ejemplo. Cuando decimos que ayuda a simplificar, nos referimos a que elimina valores que seguramente no sean significativos o que no vayan a formar parte de alguna solución. Esto ayuda a hacer más eficiente la busqueda al reducir los errores (backtracks) al ejecutar el algoritmo.

2. Defina en sus propias palabras el término “Arc Consistency”

Primero, un arco es un par de variables relacionadas por una restricción. Cuando se habla de consistencia de Arco ("Arc Consistency"), el termino se refiere a una propiedad en CSP. Esta consistencia significa y asegura que para un par de variables (X1,X2), cada valor en el dominio de X2, tiene al menos un valor compatible en el dominio de X1. Si para todo valor "x" de X1 hay un valor "y" compatible (cuando decimos compatible nos referimos a que cumplen las restricciones) en el dominio X2 el arco entre las variables es consistente.

# Task 2 - CSP con Backtracking, Beam y Local Search

En este ejercicio, implementará tres algoritmos diferentes de satisfacción de restricciones para resolver un problema de programación de exámenes para cuatro estudiantes que toman siete exámenes diferentes. El problema implica calendarizar los exámenes para los estudiantes respetando diversas limitaciones y preferencias.
Restricciones:

* Todos los exámenes deberán realizarse en días diferentes, concretamente lunes, martes y miércoles.
* Ningún estudiante deberá tener más de un examen por día.
* Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

## Configuracion de entorno para los 3 tipos de algoritmos:

In [15]:
from itertools import combinations  
import random  
import pandas as pd
import time 

# configuracion inicial
examenes = ["Calculo", "Fisica", "Algebra", "IA", "Algoritmos", "Sistemas", "Geometria"]
dias = ["Lunes", "Martes", "Miercoles"]
estudiantes = ["Nelson", "Christian", "Chuy", "Suriano"]


# definicion de variables y Dominio: Todos los exámenes pueden asignarse a cualquier dia al principio
dominio = {examen: list(dias) for examen in examenes}

print("Dominio: ", dominio)

# ejemplo de examenes por estudiante
estudiantes_examenes = {
    "Nelson": ["Calculo", "Fisica", "Algebra"],
    "Christian": ["Fisica", "IA", "Algoritmos"],
    "Chuy": ["Algebra", "Sistemas", "Geometria"],
    "Suriano": ["IA", "Algoritmos", "Sistemas"]
}

# restricciones
# Restricciones: Lista de tuplas (examen1, examen2) que no pueden estar en el mismo día
restricciones = []

# Restriccion 2: Estudiantes no pueden tener 2 exzmenes el mismo día
for estudiante, cursos in estudiantes_examenes.items():
    for curso1, curso2 in combinations(cursos, 2):
        if (curso2, curso1) not in restricciones:  # Evitar duplicados
            restricciones.append((curso1, curso2))

print("Restricciones: ", restricciones)

# ver si las asignaciones no dan problemas
def es_asignacion_valida(asignacion, restricciones):
    for exam1, exam2 in restricciones:
        if exam1 in asignacion and exam2 in asignacion:
            if asignacion[exam1] == asignacion[exam2]:
                return False
    return True

Dominio:  {'Calculo': ['Lunes', 'Martes', 'Miercoles'], 'Fisica': ['Lunes', 'Martes', 'Miercoles'], 'Algebra': ['Lunes', 'Martes', 'Miercoles'], 'IA': ['Lunes', 'Martes', 'Miercoles'], 'Algoritmos': ['Lunes', 'Martes', 'Miercoles'], 'Sistemas': ['Lunes', 'Martes', 'Miercoles'], 'Geometria': ['Lunes', 'Martes', 'Miercoles']}
Restricciones:  [('Calculo', 'Fisica'), ('Calculo', 'Algebra'), ('Fisica', 'Algebra'), ('Fisica', 'IA'), ('Fisica', 'Algoritmos'), ('IA', 'Algoritmos'), ('Algebra', 'Sistemas'), ('Algebra', 'Geometria'), ('Sistemas', 'Geometria'), ('IA', 'Algoritmos'), ('IA', 'Sistemas'), ('Algoritmos', 'Sistemas')]


### Implementación de backtracking search

In [19]:
#backtracking search
def backtracking_search(asignacion_actual, examenes_restantes, dominio, restricciones):

    if not examenes_restantes:
        return asignacion_actual.copy()  # hay solucion
    
    examen = examenes_restantes[0]
    
    for dia in dominio[examen]:
        nueva_asignacion = asignacion_actual.copy()
        nueva_asignacion[examen] = dia
        
        if es_asignacion_valida(nueva_asignacion, restricciones):
            resultado = backtracking_search(
                nueva_asignacion,
                examenes_restantes[1:],
                dominio,
                restricciones
            )
            if resultado is not None:
                return resultado
    return None  # no hubo solucion

asignacion_inicial = {}
examenes_sin_asignar = examenes.copy()

# Medir tiempo de ejecución
inicio = time.perf_counter()
solucion = backtracking_search(asignacion_inicial, examenes_sin_asignar, dominio, restricciones)
final = time.perf_counter() 

tiempo_backtracking = final - inicio

# Resultados
print("Solución encontrada:", solucion)
horario_estudiantes = {estudiante: {dia: [] for dia in dias} for estudiante in estudiantes}

# Asignar exámenes a cada estudiante según la solución
for examen, dia in solucion.items():
    for estudiante, cursos in estudiantes_examenes.items():
        if examen in cursos:
            horario_estudiantes[estudiante][dia].append(examen)

# Convertir a DataFrame de pandas
df_horario = pd.DataFrame.from_dict(horario_estudiantes, orient='index')
df_horario = df_horario[dias]  # Ordenar columnas por días

# Mostrar la tabla
print("Horario de Exámenes por Estudiante:")
print("Tiempo de ejecución:", tiempo_backtracking, "segundos")
df_horario



Solución encontrada: {'Calculo': 'Lunes', 'Fisica': 'Martes', 'Algebra': 'Miercoles', 'IA': 'Lunes', 'Algoritmos': 'Miercoles', 'Sistemas': 'Martes', 'Geometria': 'Lunes'}
Horario de Exámenes por Estudiante:
Tiempo de ejecución: 9.90999978967011e-05 segundos


,Lunes,Martes,Miercoles
Nelson,[Calculo],[Fisica],[Algebra]
Christian,[IA],[Fisica],[Algoritmos]
Chuy,[Geometria],[Sistemas],[Algebra]
Suriano,[IA],[Sistemas],[Algoritmos]


### Implementación de beam search

In [17]:
#beam search

### Implementacion de local search

In [18]:
#local search